# Various plots of NGC 4380 using VERTICO data

Isaac Cheng - September 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib.patches import Ellipse
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales
from radio_beam import Beam
%matplotlib widget

# Moment 0 Maps

## Surface density moment 0 map

In [2]:
# Surface density moment 0 map
imgpath = "/arc/projects/vertico/products/release.v1.2/native/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_mom0_Msun.fits"
# 
# Extract image data
# 
# imgdata = fits.getdata(imgpath)
with fits.open(imgpath) as hdu_list:
    hdu_list.info()
    # Get image data (typically in PRIMARY block)
    imgdata = hdu_list[0].data  # 2D array
    # Get image coordinates
    imgheader = hdu_list[0].header
    imgwcs = WCS(hdu_list[0].header)
    # print(hdu_list[0].header)  # astropy strips some keywords
                                 # https://docs.astropy.org/en/stable/io/fits/usage/headers.html
# Trim background
imgdata[imgdata <= 0] = np.nan  # background value is zero
# print(imgwcs.pixel_to_world(0,0))
# print(imgwcs.pixel_to_world(*imgdata.shape))

Filename: /arc/projects/vertico/products/release.v1.2/native/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_mom0_Msun.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      63   (78, 78)   float64   


In [3]:
# Plot image data
levels = 22  # number of contour/cmap levels
cmap = mpl.cm.get_cmap("inferno_r", levels)
fig, ax = plt.subplots(subplot_kw={"projection": imgwcs})
# ! DO NOT TRANSPOSE DATA !
# img = ax.imshow(imgdata, cmap=cmap, vmin=0, interpolation="none")
img = ax.contourf(imgdata, levels=levels, cmap=cmap, vmin=0)
# ax.contour(imgdata, levels=levels, colors="w", linewidths=0.01)
cbar = fig.colorbar(img)
cbar.ax.minorticks_off()
cbar.set_ticks([0, 10, 20, 30])
cbar.set_label(r"Surface Density ($\rm M_\odot\; pc^{-2}$)")
# cbar.ax.tick_params(direction="out")
# ax.tick_params(which="both", direction="out")
# 
# Add 1 kpc scale bar
# 
dist = 16.5 * u.Mpc
arcsec_per_px = (proj_plane_pixel_scales(imgwcs.celestial)[0] * u.deg).to(u.arcsec)
# Still deciding whether to use arctan for the following
arcsec_per_kpc = np.rad2deg(1 * u.kpc / dist.to(u.kpc) * u.rad).to(u.arcsec)
px_per_kpc = arcsec_per_kpc / arcsec_per_px
scalebar = AnchoredSizeBar(ax.transData, px_per_kpc, label="1 kpc", loc="lower right",
                           pad=1, color="k", frameon=False, size_vertical=0.5)
ax.add_artist(scalebar)
# 
# Add beam size
# 
deg_per_px = proj_plane_pixel_scales(imgwcs.celestial)[0] * u.deg
beam = Beam.from_fits_header(imgheader)
xbeam, ybeam = (8, 9)
ellipse = Ellipse(
    xy=(xbeam, ybeam),
    width=(beam.major.to(u.deg) / deg_per_px).to(u.dimensionless_unscaled).value,
    height=(beam.minor.to(u.deg) / deg_per_px).to(u.dimensionless_unscaled).value,
    # PA is 90 deg offset from x-y axes by convention,  (it is angle from NCP)
    angle=(beam.pa + 90 * u.deg).to(u.deg).value,
    ls="-", edgecolor="k", fc='None', lw=2, zorder=2
)
ax.add_patch(ellipse)
# Add text label
xtxt, ytxt = (xbeam - 4, ybeam - 5)
ax.text(xtxt, ytxt, "Beam")
# 
# Other plot parameters
# 
ax.grid(False)
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.set_title("NGC 4380: Moment 0 Surface Density Map")
ax.set_aspect("equal")
fig.savefig("NGC4380_mom0_Msun_v2.pdf")
plt.show()
# TODO: fix contours to match pre-existing plot?
# TODO: add outline to determine "edge" of galaxy?

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
plt.close("all")